In [1]:
# -*- coding: utf-8 -*
import twitter
import csv
import json
import re
import requests
import lxml.html as html
from urllib2 import urlopen
import pandas as pd
from datetime import datetime, timedelta
from dateutil import tz
import time
import sklearn.cross_validation as cv
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import numpy as np
import xml.etree.ElementTree as ET
%matplotlib inline

In [2]:
OUT_NEWS_FILE = "news.csv"
OUT_TWITTER_FILE = "twitter.csv"

In [3]:
class TJLoader:

    def __init__(self):
        self._news_pages = ["https://tjournal.ru/paper/page/{}"]#, "https://tjournal.ru/club/news/recent/page/{}"]
        self._month_map = {u"января":"01", u"февраля":"02", u"марта":"03", u"апреля":"04", u"мая":"05", u"июня":"06", u"июля":"07", u"августа":"08", u"сентября":"09", u"октября":"10", u"ноября":"11", u"декабря":"12"}

    def get_news_uri(self, min_index=10, count=30):
        """

        :param min_index: int, индекс страницы, с которой нужно начать поиск
        :param count: int, количество страниц, которые нужно скачать
        :return: list. список ссылок на новости
        """
        links = []
        for news_page in self._news_pages:
            
            for i in range(count):
                page = html.parse(urlopen(news_page.format(i+min_index)))
                divs = page.getroot().find_class('b-articles__b__title')

                for div in divs:
                    links.append(div.getchildren()[1].get("href"))
                    
        return links

    
    def _parse_date(self, date):
        date = date.replace(",", "")
        date = date.split(" ")
        
        converted_date = date[2]
        converted_date +="-"+self._month_map[date[1]]
        converted_date +="-"+date[0]
        
        converted_date +=" "+date[3]
        
        return converted_date
        

    def get_link_info(self, link):
        """

        :param link: str, url страницы с tjournal, для которой нужно собрать информацию
        :return: dict с данными со страницы
        """
        page = html.parse(urlopen(link))
        root = page.getroot()

        # заголовок
        title = root.find_class("b-article__title")
        title = title[0].find("h1").text

        # парсим количество просмотров
        view = root.get_element_by_id("hitsCount").text
        view = view.replace(" ", "")
        view = int(view)

        # Количество комментариев
        comments = root.find_class("b-article__infoline__comments")
        comment = int(comments[0].find("b").text.replace(" ", ""))

        # Теги
        tags = root.find_class("b-article__tags__tag")
        tag_list = []
        for tag in tags:
            tag_list.append(tag.text)
        
        # Дата
        date = root.find_class("b-article__infoline__date")
        date = self._parse_date(date[0].text)
        
        # Первоисточник
        source = root.find_class("b-article__link")
        if len(source) != 0:
            source = source[0].getchildren()[0].getchildren()[1].getchildren()[0].text
        else:
            source = None
        
        # Type
        if "tjournal.ru/c/" in link:
            news_type = "TJ_C"
        else:
            news_type = "TJ_P"
            
        return {
            "url": link, 
            "title": title, 
            "views": view, 
            "comments": comment, 
            "tags": tag_list, 
            "date": date, 
            "type": news_type, 
            "source": source}


    def get_tj_news_info(self, min_index=1, count=30, first_date="2017-01-01", last_date="2010-01-01"):
        """
        :param min_index: int, индекс минимальной страницы, откуда начинаем поиск
        :param count: int, количество страниц, по которым ищем
        :first_date: время первой новости, которую мы скачаем
        :last_date: время последней новости, которую мы скачаем
        :return: dict с данными со страницы
        """
        links = self.get_news_uri(min_index=min_index, count=count)
        link_info_list = []
        i = 0
        is_break = False
        for link in links:
            link_info = self.get_link_info(link)
            # Если заданное время не подходит
            if link_info["date"] > first_date or link_info["date"] < last_date:
                continue
    
            link_info_list.append(link_info)
            
            i+=1
            if i%10 == 0:
                print "Скачали ", i, " страниц"

        return link_info_list

In [4]:
class VCLoader:

    def __init__(self):
        self._news_pages = "https://api.vc.ru/1/paper"
        self._month_map = {u"января":"01", u"февраля":"02", u"марта":"03", u"апреля":"04", u"мая":"05", u"июня":"06", u"июля":"07", u"августа":"08", u"сентября":"09", u"октября":"10", u"ноября":"11", u"декабря":"12"}

    def get_news_uri(self, min_index=10, count=30):
        """

        :param min_index: int, индекс страницы, с которой нужно начать поиск
        :param count: int, количество страниц, которые нужно скачать
        :return: list. список ссылок на новости
        """
        links = []
        text = requests.get(self._news_pages).text
        json_req = json.loads(text)
    
        for news in json_req:
            links.append(news["url"])

        return links

    
    def _parse_date(self, date):
        date = date.replace(",", "")
        date = date.split(" ")
        
        converted_date = date[2]
        converted_date +="-"+self._month_map[date[1]]
        converted_date +="-"+date[0]
        
        converted_date +=" "+date[3]
        
        return converted_date
        

    def get_link_info(self, link):
        """

        :param link: str, url страницы с vc, для которой нужно собрать информацию
        :return: dict с данными со страницы
        """
        page = html.parse(urlopen(link))
        root = page.getroot()

        # заголовок
        title = root.find_class("b-article__head")
        title = title[0].find("h1").text

        # парсим количество просмотров
        view = root.get_element_by_id("hitsCount").text
        view = view.replace(" ", "")
        view = int(view)

        # Количество комментариев
        comments = root.find_class("ccount")[0].text
        comment = int(comments.replace(" ", ""))

        # Теги
        tags = root.find_class("b-tags__tag")
        tag_list = []
        for tag in tags:
            tag_list.append(tag.text)
        
        # Дата
        date = root.find_class("b-article__infopanel__date")
        date = self._parse_date(date[0].text)
        

        return {"title": title, "views": view, "comments": comment, "tags": tag_list, "date": date, "url": link, "type": "VC"}


    def get_tj_news_info(self, min_index=1, count=30, first_date="2010-01-01", last_date="2017-01-01"):
        """
        :param min_index: int, индекс минимальной страницы, откуда начинаем поиск
        :param count: int, количество страниц, по которым ищем
        :first_date: str, дата и время первой (самой новой) новости
        :last_date: str, дата и время последней(самой старой) новости
        
        :return: dict с данными со страницы
        """
        links = self.get_news_uri(min_index=min_index, count=count)
        link_info_list = []
        i = 0
        
        for link in links:
            
            link_info = self.get_link_info(link)
            # Если заданное время не подходит
            if link_info["date"] > first_date or link_info["date"] < last_date:
                continue
            
            link_info_list.append(link_info)
            
            i+=1
            if i%10 == 0:
                print "Скачали ", i, " страниц"

        return link_info_list

In [5]:
class RSSLoader:
    
    def __init__(self):

        self._pages = ["https://roem.ru/rss/roem-all-news.xml", "http://lifenews.ru/xml/feed.xml", "http://www.forbes.ru/newrss.xml", "http://www.vesti.ru/vesti.rss", "http://lenta.ru/rss", "http://ria.ru/export/rss2/index.xml", "https://navalny.com/blog/post.rss", "https://slon.ru/export/all.xml", "https://meduza.io/rss/all", "http://www.vedomosti.ru/rss/news"]
        self._month_dict = {"Jan":"1", "Feb":"2", "Mar":"3", "Apr":"4", "May":"5", "Jun":"6", "Jul":"7", "Aug":"8", "Sep":"9", "Oct":"10", "Nov":"11", "Dec":"12"}
        
        self._UTC_TIME_ZONE = tz.gettz('Europe/London')
        self._MOSCOW_TIME_ZONE = tz.gettz('Europe/Moscow')
        self._RATE_LIMIT = "[{u'message': u'Rate limit exceeded', u'code': 88}]"
        
        
    def _parse_date(self, date):

        """

        :param date: str, дата в формате - "Fri, 04 Dec 2015 14:45:39 +0000"
        :return: str, дата в человеческом, но буржуйском формате, да еще и в Московском часовом поясе
        """
        timezone = date.split("+")[1]
        date = date.replace(",", "")
        date_array = date.split(' ')
        
        time = date_array[4]
        day = int(date_array[1])
        month = self._month_dict[date_array[2]]
        year = date_array[3]

        date = str(year)+"-"+str(month)+"-"+str(day)+" "+time
        date = datetime.strptime(date, '%Y-%m-%d %H:%M:%S')
        
        if (timezone != "0300"):
            utc_date = date.replace(tzinfo=self._UTC_TIME_ZONE)
            date = utc_date.astimezone(self._MOSCOW_TIME_ZONE)

        # Обрезаем зону
        date = str(date).split("+")[0]
        
        # Обрезаем секунды
        splited = date.split(":")
        date = splited[0]+":"+splited[1]
        return date
    
    
    def _get_type(self, url):
        url = url.replace("www.", "")
        url = url.split("://")[1]
        url = url.split("/")[0]
        return url
    
    def _handle_data(self, url, link, title, date):
        
        # Убрать лишние пробелы
        title = re.sub(' +',' ', title)
        
        if url=="https://slon.ru/export/all.xml":
            link = link.split("?")[0]
            title = title.replace("\n", "")
            
        # Приведение даты к общему формату и временной зоне
        date = self._parse_date(date)
        
        return link, title, date
        
        
    
    def _get_hours_until_now(self, date):
        news_date = datetime.strptime(date, '%Y-%m-%d %H:%M')
        now = datetime.today()
        return float((now-news_date).total_seconds()/3600)
        
    def get_news_array(self, hour=5):
        """
        hour: int, время последней новости
        """
        news_array = []
        for url in self._pages:
            try:
                news_type = self._get_type(url)
                print url

                tree = ET.ElementTree(file=urlopen(url))
                root = tree.getroot()
                for i in root.iter('item'):

                    link = i.find('link').text
                    title = i.find('title').text
                    date = i.find('pubDate').text

                    link, title, date = self._handle_data(url, link, title, date)
                    news_info =  {
                        "title": title,
                        "date": date, 
                        "url": link, 
                        "type": news_type
                    }

                    if self._get_hours_until_now(date) >= hour:
                        news_array.append(news_info)
            except Exception:
                print "Url:" + url + " упал"

                
            
        print "Собрано ", len(news_array), " новостей с RSS"
        return news_array

In [65]:
class TwitterLoader:


    def __init__(self):
        CONSUMER_KEY = 'BOuuaMDhNhm6yx0rzqK8bMsbI'
        CONSUMER_SECRET = '3DybJwlkXd2vU6R385yLA8yJblYJltLtwojySD9AVs04ShauZ0'

        ACCESS_TOKEN_KEY = '3712177576-of3jzZ8gNmlPDfPjPyR0Ljw1Ao2IXdTqX9dZGDZ'
        ACCESS_TOKEN_SECRET = 'Ky7iKwByHNXX3UMfuMhv6UgVx2IhjLo3KmwpsBQz35wtG'

        self.api = twitter.Api(consumer_key=CONSUMER_KEY,
                  consumer_secret=CONSUMER_SECRET,
                  access_token_key=ACCESS_TOKEN_KEY,
                  access_token_secret=ACCESS_TOKEN_SECRET)
        self._month_dict = {"Jan":"1", "Feb":"2", "Mar":"3", "Apr":"4", "May":"5", "Jun":"6", "Jul":"7", "Aug":"8", "Sep":"9", "Oct":"10", "Nov":"11", "Dec":"12"}

        self._UTC_TIME_ZONE = tz.gettz('Europe/London')
        self._MOSCOW_TIME_ZONE = tz.gettz('Europe/Moscow')
        self._RATE_LIMIT = "[{u'message': u'Rate limit exceeded', u'code': 88}]"



    def _parse_date(self, date):

        """

        :param date: str, дата в формате твиттера - "Sat Nov 21 17:00:29 +0000 2015"
        :return: str, дата в человеческом, но буржуйском формате, да еще и в Московском часовом поясе
        """
        date_array = date.split(' ')
        month = self._month_dict[date_array[1]]
        day = int(date_array[2])
        time = date_array[3]
        year = date_array[5]

        date = str(year)+"-"+str(month)+"-"+str(day)+" "+time
        date = datetime.strptime(date, '%Y-%m-%d %H:%M:%S')
        utc_date = date.replace(tzinfo=self._UTC_TIME_ZONE)
        moscow_date = utc_date.astimezone(self._MOSCOW_TIME_ZONE)

        return str(moscow_date).split("+")[0]



    def loadTweetWithLink(self, link, date):
        """

        :param link: str, ключевое слово для поиска, вданном случае - ссылка на письмо
        :param date: str, дата, до которой искать твиты
        :return: список словариков с информацией о УНИКАЛЬНЫХ твитах по запросу link
        """
        set_id = set()
        tweet_list = []
        result = self.api.GetSearch(term=link, until=date, count=100000)

        for res in result:
            print res.GetCreatedAt()
            tw_id = res.GetId()
            
            # Если мы уже обрабатывали этот твит - идем дальше
            if tw_id in set_id:
                print "========="
                print "countinue!!!"
                print "========="
                continue
            
            retweeted_status = res.GetRetweeted_status()
            # Если это не ретвит
            if retweeted_status is None:
                is_retweet = 0
                retweeted_count = res.GetRetweetCount()
                favorite_count = res.GetFavoriteCount()
            else:
                is_retweet = 1
                retweeted_count = 0
                favorite_count = 0
            set_id.add(tw_id)

            created_at = res.GetCreatedAt()
            created_at = self._parse_date(created_at)

            # Данные о пользователе
            user = res.GetUser()
            followers_count = user.followers_count
            listed_count = user.listed_count
            friends_count = user.friends_count
            favourites_count = user.favourites_count
            statuses_count = user.statuses_count
            print tw_id


            tw_dict= {
                    "url": link,
                    "tw_id":tw_id,
                    "retweeted_count": retweeted_count,
                    "favorite_count":favorite_count,
                    "is_retweet": is_retweet,
                    "created_at":created_at,
                    "user_followers_count":followers_count,
                    "user_listed_count": listed_count,
                    "user_friends_count":friends_count,
                    "user_favourites_count":favourites_count,
                    "user_statuses_count":statuses_count
                    }

            tweet_list.append(tw_dict)

        return tweet_list

    def _get_next_date(self, date, days):
        """

        :param date: str, дата
        :param days: int, количество дней
        :return: возвращает дату через days-дней после date
        """
        date = datetime.strptime(date, '%Y-%m-%d %H:%M')
        date += timedelta(days=days)
        return str(date).split(' ')[0]


    def load_tweets_by_term(self, news_list, days_after_news=2):
        """

        :param news_list: list(str), список словариков с информацией о новости
        :param days_after_news:  int, количество дней после публикации новости, до которой искать
        """
        result_list = []
        i = 0
        for news in news_list:
            tweets = []
            try:
                until_date = self._get_next_date(news["date"], days_after_news)
                print until_date
                tweets = self.loadTweetWithLink(news["url"], until_date)
            except twitter.error.TwitterError as ex:
                print str(ex)
                if str(ex) == self._RATE_LIMIT:
                    sleep_time = self.api.GetSleepTime("search/tweets") #??? Почему-то не работает
                    print "Спим {} сек.".format(sleep_time)
                    time.sleep(sleep_time+2)
                    tweets = self.loadTweetWithLink(news["url"], until_date)

            result_list+= tweets
            
            i+=1
            if i%10 == 0:
                print "Собрано информация о", i, " новостях"
            
        return result_list

### Подготавливаем  данные для анализа

In [152]:
# Заметка: Фильтровать данные по дате! Вычеркнуть первый день, чтобы не портить данные
OUT_TWITTER_FILE = "StreamingTest/tweets.csv"
OUT_NEWS_FILE = "StreamingTest/news.csv"

In [153]:
# Время для вычисления независимых переменных
FIRST_TIME = 9
# Время для вычисления целевой функции
LAST_TIME = 180

In [154]:
news_info = pd.read_csv(OUT_NEWS_FILE, sep=",")
tweeter_data = pd.read_csv(OUT_TWITTER_FILE, sep=",")

In [155]:
print tweeter_data.created_at.min()
print tweeter_data.created_at.max()

2015-12-12 02:52:57
2015-12-17 18:41:20


In [156]:
news_info = news_info[news_info["news_date"] > '2015-12-12 06:00']
news_info = news_info[news_info["news_date"] < '2015-12-17 10:00']

In [157]:
# !!!!   !!!!!!!!!! !!!!!!!!!!1 !!!!!!!!!!!!
# Сначала почищу данные. Там, где не материалы редакции - мусор, на который твитов то почти и нет!
#news_info = news_info[ (news_info['type'] != "TJ_C")]
len(news_info)

5254

In [158]:
# Мерджим дата фреймы
df = news_info.merge(tweeter_data, on='url', left_index=True, right_index=False)

In [159]:
def diff_date_minutes(news_date, tweet_date):
    news_date = datetime.strptime(news_date, '%Y-%m-%d %H:%M')
    tweet_date = datetime.strptime(tweet_date, '%Y-%m-%d %H:%M:%S')
    return int((tweet_date-news_date).total_seconds()/60)

In [160]:
# количество минут, с момента публикации записи, данного твита
df["time_since_news"] = df.apply(lambda s: diff_date_minutes(s["news_date"], s["created_at"]), axis=1)

In [161]:
def get_week_day(date):
    date = datetime.strptime(date, '%Y-%m-%d %H:%M')
    return date.weekday()

In [162]:
# индекс дня недели
df["week_day_news"] = df.news_date.apply(lambda s: get_week_day(s))

In [163]:
def get_minutes_since_midnight(date):
    midnight = date.split(" ")[0] + " 00:00"
    date = datetime.strptime(date, '%Y-%m-%d %H:%M')
    midnight = datetime.strptime(midnight, '%Y-%m-%d %H:%M')
    return int((date-midnight).total_seconds()/60)

In [164]:
# Сколько минут времени прошло с полуночи
df["minutes_since_midnight"] = df.news_date.apply(lambda s: get_minutes_since_midnight(s))

In [165]:
# Есть новости с ошибочной датой. Их исключаем
df = df[df["time_since_news"] > 0]

In [166]:
news_info["type"].unique()

array(['VC', 'lifenews.ru', 'vesti.ru', 'lenta.ru', 'ria.ru', 'meduza.io',
       'vedomosti.ru', 'roem.ru', 'forbes.ru', 'navalny.com', 'slon.ru'], dtype=object)

### Агрегируем данные из твиттера

In [167]:
len(df["url"].unique())

2961

In [168]:
ft_df = df[df["time_since_news"] <= FIRST_TIME]

In [169]:
grouped = ft_df.groupby("url")

In [170]:
# Считаем общее количество твиттов
count_of_tweets = pd.DataFrame(grouped["url"].count())
count_of_tweets.columns = ["first_time_tweet"]
count_of_tweets.reset_index(inplace=True)  
df = pd.merge(df, count_of_tweets, on='url', left_index=True, right_index=False, how="outer")

In [171]:
# Считаем общую аудиторию
follower_sum = pd.DataFrame(grouped["user_followers_count"].sum())
follower_sum.columns = ["follower_sum"]
follower_sum.reset_index(inplace=True)  
df = pd.merge(df, follower_sum, on='url', left_index=True, right_index=False, how="outer")

In [172]:
# Считаем число ретвитов
retweeted_count_sum = pd.DataFrame(grouped["is_retweet"].sum())
retweeted_count_sum.columns = ["retweeted_count_sum"]
retweeted_count_sum.reset_index(inplace=True)  
df = pd.merge(df, retweeted_count_sum, on='url', left_index=True, right_index=False, how="outer")

In [173]:
# Считаем число звездочек
"""
favorite_count_sum = pd.DataFrame(grouped["favorite_count"].sum())
favorite_count_sum.columns = ["favorite_count_sum"]
favorite_count_sum.reset_index(inplace=True)  
df = pd.merge(df, favorite_count_sum, on='url', left_index=True, right_index=False, how="outer")
""";

In [174]:
# Считаем общее число списков, в которых состоят сделавшие посты
user_listed_count = pd.DataFrame(grouped["user_listed_count"].sum())
user_listed_count.columns = ["user_listed_count_sum"]
user_listed_count.reset_index(inplace=True)  
df = pd.merge(df, user_listed_count, on='url', left_index=True, right_index=False, how="outer")

In [175]:
st_df = df[df["time_since_news"] <= LAST_TIME]

In [176]:
# Выделяем значение целевой функции
grouped = st_df.groupby("url")
count_of_tweets = pd.DataFrame(grouped["url"].count())
count_of_tweets.columns = ["last_time_tweet"]
count_of_tweets.reset_index(inplace=True)  
df = pd.merge(df, count_of_tweets, on='url', left_index=True, right_index=False, how="outer")

In [177]:
# Бинаризируем категориальный признак type
df.reset_index(inplace=True)
dummy = pd.get_dummies(df['type'])
df = df.join(dummy)

In [178]:
PREPARED_CSV = "prepared_to_analys.csv"

In [179]:
# Сохраняемся
df.to_csv(PREPARED_CSV, sep=",", index=False, encoding="utf-8", quoting=csv.QUOTE_NONNUMERIC)

In [151]:
df.columns

Index([u'index', u'comments', u'date', u'news_date', u'source', u'tags',
       u'title', u'type', u'url', u'views', u'created_at', u'is_retweet',
       u'screen_name', u'tw_id', u'user_favourites_count',
       u'user_followers_count', u'user_friends_count', u'user_listed_count',
       u'user_statuses_count', u'time_since_news', u'week_day_news',
       u'minutes_since_midnight', u'first_time_tweet', u'follower_sum',
       u'retweeted_count_sum', u'user_listed_count_sum', u'last_time_tweet',
       u'VC', u'forbes.ru', u'lenta.ru', u'lifenews.ru', u'meduza.io',
       u'navalny.com', u'ria.ru', u'roem.ru', u'slon.ru', u'vedomosti.ru',
       u'vesti.ru'],
      dtype='object')

In [61]:
#print df.columns
df_features = ["url", "news_date", "created_at","week_day_news", "minutes_since_midnight", "first_time_tweet","follower_sum", "retweeted_count_sum", "user_listed_count_sum", "last_time_tweet", 'VC', 'forbes.ru', 'lenta.ru', 'lifenews.ru', 'meduza.io', 'navalny.com', 'ria.ru', 'roem.ru', 'slon.ru', 'vedomosti.ru', 'vesti.ru']
df = df[df_features]
df = df.drop_duplicates().dropna()
df.reset_index(inplace=True, drop=True)

In [186]:
a = df.sort_values('created_at')
a[a["url"] == "http://lifenews.ru/news/174729"]

,url,news_date,created_at,week_day_news,minutes_since_midnight,first_time_tweet,follower_sum,retweeted_count_sum,user_listed_count_sum,last_time_tweet,...,forbes.ru,lenta.ru,lifenews.ru,meduza.io,navalny.com,ria.ru,roem.ru,slon.ru,vedomosti.ru,vesti.ru
7874,http://lifenews.ru/news/174729,2015-12-12 06:25,2015-12-12 06:26:31,5,385,19,553969,15,3387,56,...,0,0,1,0,0,0,0,0,0,0
7875,http://lifenews.ru/news/174729,2015-12-12 06:25,2015-12-12 06:26:32,5,385,19,553969,15,3387,56,...,0,0,1,0,0,0,0,0,0,0
7876,http://lifenews.ru/news/174729,2015-12-12 06:25,2015-12-12 06:27:30,5,385,19,553969,15,3387,56,...,0,0,1,0,0,0,0,0,0,0
7877,http://lifenews.ru/news/174729,2015-12-12 06:25,2015-12-12 06:27:34,5,385,19,553969,15,3387,56,...,0,0,1,0,0,0,0,0,0,0
7878,http://lifenews.ru/news/174729,2015-12-12 06:25,2015-12-12 06:27:36,5,385,19,553969,15,3387,56,...,0,0,1,0,0,0,0,0,0,0
7879,http://lifenews.ru/news/174729,2015-12-12 06:25,2015-12-12 06:27:37,5,385,19,553969,15,3387,56,...,0,0,1,0,0,0,0,0,0,0
7880,http://lifenews.ru/news/174729,2015-12-12 06:25,2015-12-12 06:27:38,5,385,19,553969,15,3387,56,...,0,0,1,0,0,0,0,0,0,0
7881,http://lifenews.ru/news/174729,2015-12-12 06:25,2015-12-12 06:27:43,5,385,19,553969,15,3387,56,...,0,0,1,0,0,0,0,0,0,0
7882,http://lifenews.ru/news/174729,2015-12-12 06:25,2015-12-12 06:27:48,5,385,19,553969,15,3387,56,...,0,0,1,0,0,0,0,0,0,0
7883,http://lifenews.ru/news/174729,2015-12-12 06:25,2015-12-12 06:27:52,5,385,19,553969,15,3387,56,...,0,0,1,0,0,0,0,0,0,0


In [209]:
sum(df["favorite_count_sum"].isnull())

5032

# После первого запуска, через несколько дней, запускать только часть, которая находится под этим заголовком!

In [7]:
HOURS = 8 # Количество часов от текущего момента. Текущее время - Hours часов - время последней собранной новости

In [8]:
def get_hours_ago(hours=3):
    d = datetime.now() - timedelta(hours=hours)
    d = str(d).split(":")
    d = d[0]+":"+d[1]
    return d


def load_new_news(news_df, hours=3):
    # Вычисляем самую молодую новость
    tj_last_date = (news_df[news_df["type"]=="TJ_P"]["date"]).max()
    #tj_c_last_date = (news_df[news_df["type"]=="TJ_C"]["date"]).max()
    
    #tj_last_date = max(tj_p_last_date, tj_c_last_date)
    vc_last_date = (news_df[news_df["type"]=="VC"]["date"]).max()
    first_date = get_hours_ago(hours)
    
    
    print "Качаем новости с ", tj_last_date," и ", vc_last_date, " по ", first_date
    
    tj_loader = TJLoader()
    tj_pages = tj_loader.get_tj_news_info(min_index=0, count=4, first_date=first_date, last_date=tj_last_date)    
    
    vc_loader = VCLoader()
    vc_pages = vc_loader.get_tj_news_info(first_date=first_date, last_date=vc_last_date)
    
    pages = vc_pages + tj_pages
    pages
    
    return pages

# Загружаем данные
news_prev_df = pd.read_csv(OUT_NEWS_FILE, sep=",")
twitter_prev_df = pd.read_csv(OUT_TWITTER_FILE, sep=",")

# Качаем новости с VC и TJ
pages = load_new_news(news_prev_df, HOURS)
# Качаем новости RSS
rss_loader = RSSLoader()
rss_pages = rss_loader.get_news_array()
# Объединяем
pages += rss_pages

tw = TwitterLoader()
tweets = tw.load_tweets_by_term(pages)

# Объединяем новости с предыдущими
news = pd.DataFrame(pages)
news = (news.append(news_prev_df)).reset_index(drop=True)

# Объединяем твиты с предыдущими
tweets_df = pd.DataFrame(tweets)
tweets_df = (tweets_df.append(twitter_prev_df)).reset_index(drop=True)

# Сохраняем
tweets_df.to_csv(OUT_TWITTER_FILE, sep=",", index=False, encoding="utf-8", quoting=csv.QUOTE_NONNUMERIC)
news.to_csv(OUT_NEWS_FILE, sep=",", index=False, encoding="utf-8", quoting=csv.QUOTE_NONNUMERIC)

Качаем новости с  2015-12-06 00:21  и  2015-12-05 18:45  по  2015-12-09 11:01
Скачали  10  страниц
Скачали  20  страниц
Скачали  30  страниц
Скачали  40  страниц
Скачали  10  страниц
Скачали  20  страниц
Скачали  30  страниц
https://roem.ru/rss/roem-all-news.xml
http://lifenews.ru/xml/feed.xml
http://www.forbes.ru/newrss.xml
http://www.vesti.ru/vesti.rss
http://lenta.ru/rss
http://ria.ru/export/rss2/index.xml
https://navalny.com/blog/post.rss
https://slon.ru/export/all.xml
https://meduza.io/rss/all
http://www.vedomosti.ru/rss/news
Собрано  488  новостей с RSS
Собрано информация о 10  новостях
Собрано информация о 20  новостях
Собрано информация о 30  новостях
Собрано информация о 40  новостях
Собрано информация о 50  новостях
Собрано информация о 60  новостях
Собрано информация о 70  новостях
Собрано информация о 80  новостях
Собрано информация о 90  новостях
Собрано информация о 100  новостях
Собрано информация о 110  новостях
Собрано информация о 120  новостях
Собрано информация о 13

# УДАЛЯЕМ ПОВТОРЫ =(((

In [9]:
# Сначала для твиттера
tw_data = pd.read_csv(OUT_TWITTER_FILE, sep=",")
last_size = len(tw_data)
dupl = tw_data["tw_id"].duplicated()
dupl = np.invert((dupl.as_matrix()))
tw_data = tw_data[dupl]
print "Удалено ", last_size-len(tw_data)

Удалено  566


In [10]:
# Сохраняем!
tw_data.to_csv(OUT_TWITTER_FILE, sep=",", index=False, encoding="utf-8", quoting=csv.QUOTE_NONNUMERIC)

In [11]:
# Теперь для новостей
news_df = pd.read_csv(OUT_NEWS_FILE, sep=",")
last_size = len(news_df)
dupl = news_df["url"].duplicated()
dupl = np.invert((dupl.as_matrix()))
news_df = news_df[dupl]
print "Удалено ", last_size-len(news_df)

Удалено  28


In [12]:
# Сохраняем
news_df.to_csv(OUT_NEWS_FILE, sep=",", index=False, encoding="utf-8", quoting=csv.QUOTE_NONNUMERIC)

* Ограничение на максимальное количество твиттов
* Число ретвитов уже не чистое

# ======================================

In [76]:
df["type"].unique()

array(['VC', 'TJ_P'], dtype=object)

# Здесь уже нет ничего хорошего. Уходи

In [117]:
tw_data = pd.read_csv(OUT_TWITTER_FILE, sep=",")

In [120]:
tw_data[tw_data["url"] == "https://navalny.com/p/4591/"]

,created_at,favorite_count,is_retweet,retweeted_count,tw_id,url,user_favourites_count,user_followers_count,user_friends_count,user_listed_count,user_statuses_count
12921,2015-12-07 15:07:38,180,0,555,673836259572383744,https://navalny.com/p/4591/,802,1273995,883,6918,90847
12922,2015-12-09 00:49:16,0,0,0,674345021068460032,https://navalny.com/p/4591/,0,20,24,0,1028
12923,2015-12-09 00:13:49,0,1,0,674336101339910145,https://navalny.com/p/4591/,2,12,49,0,20
12924,2015-12-08 23:39:23,0,1,0,674327432942460928,https://navalny.com/p/4591/,369,61,135,3,5252
12925,2015-12-08 23:02:38,0,1,0,674318187266183169,https://navalny.com/p/4591/,4,45,57,0,332
12926,2015-12-08 22:20:47,0,1,0,674307653917663232,https://navalny.com/p/4591/,2,61,167,0,2313
12927,2015-12-08 22:17:19,0,0,0,674306779979890689,https://navalny.com/p/4591/,144,666,435,4,6650
12928,2015-12-08 21:45:34,0,1,0,674298793219747840,https://navalny.com/p/4591/,534,2916,2914,17,4957
12929,2015-12-08 20:20:38,0,0,0,674277418215936001,https://navalny.com/p/4591/,0,7,18,1,2463
12930,2015-12-08 20:18:31,0,1,0,674276886407720964,https://navalny.com/p/4591/,366,66,285,0,715


In [50]:
news_df = pd.read_csv(OUT_NEWS_FILE, sep=",")

In [40]:
news_df["type"].unique()

array(['VC', 'TJ_P', 'roem.ru', 'lifenews.ru', 'vesti.ru', 'lenta.ru',
       'navalny.com', 'slon.ru', 'meduza.io', '.forbes.ru', '.vesti.ru',
       'TJ_C'], dtype=object)

In [ ]:
news = news.append(news_df)

In [ ]:
# читаем данные
news_df = pd.read_csv(OUT_NEWS_FILE, sep=",")
twitter_df = pd.read_csv(OUT_TWITTER_FILE, sep=",")

In [ ]:
# находим самую "молодую новость"
news_df = news_df.sort_values(by=["date"], ascending=False)
tj_last_date = (news_df[news_df["type"]=="TJ_P"]["date"]).max()
vc_last_date = (news_df[news_df["type"]=="VC"]["date"]).max()

In [ ]:
vc_last_date

In [ ]:
def get_tj_news_info(self, min_index=1, count=30, first_date="2015-11-25 12:58", last_date="2015-11-29 12:58"):

In [ ]:
# скачивае информацию о новостях
loader = TJLoader()
pages = loader.get_tj_news_info(min_index=0, count=30, first_date="2015-11-25 12:58", last_date="2015-11-29 12:58")

In [ ]:
len(pages)

In [ ]:
# Сохраняем в файлик
news_df = news_df.append(pd.DataFrame(pages))
news_df.to_csv(OUT_NEWS_FILE, sep=",", index=False, encoding="utf-8", quoting=csv.QUOTE_NONNUMERIC)

In [ ]:
# Скачиваем данные из твиттера
tw = TwitterLoader()
tweeter_data = tw.load_tweets_by_term(pages)

In [ ]:
len(tweeter_data)

In [ ]:
# Сохраняем в файлик
twitter_df = twitter_df.append(pd.DataFrame(tweeter_data))
twitter_df.to_csv(OUT_TWITTER_FILE, sep=",", index=False, encoding="utf-8", quoting=csv.QUOTE_NONNUMERIC)

In [66]:
pages = [{"url": "https://tjournal.ru/p/lesya-echo-off", 
            "date": '2015-12-05 12:40'}]

In [73]:
tw = TwitterLoader()
tweets = tw.load_tweets_by_term(pages)

2015-12-07
Sat Dec 05 09:41:02 +0000 2015
673074590814486528
Sat Dec 05 12:50:23 +0000 2015
673122243686375424
Sun Dec 06 19:41:27 +0000 2015
673588079320555522
Sun Dec 06 19:05:11 +0000 2015
673578954788364289
Sun Dec 06 18:46:03 +0000 2015
673574138183868426
Sun Dec 06 16:37:16 +0000 2015
673541729098813440
Sun Dec 06 15:12:10 +0000 2015
673520311183241216
Sun Dec 06 14:14:18 +0000 2015
673505751265079300
Sun Dec 06 12:52:05 +0000 2015
673485057705816064
Sun Dec 06 10:56:43 +0000 2015
673456027942154240
Sun Dec 06 09:03:35 +0000 2015
673427555232387072
Sun Dec 06 09:01:49 +0000 2015
673427111571517440
Sun Dec 06 08:50:36 +0000 2015
673424287743746049
Sun Dec 06 07:34:43 +0000 2015
673405191622905856
Sun Dec 06 07:31:06 +0000 2015
673404282176192512
Sun Dec 06 07:15:11 +0000 2015
673400274522910720
Sun Dec 06 06:18:26 +0000 2015
673385995266605056
Sun Dec 06 05:56:15 +0000 2015
673380412123701248
Sun Dec 06 05:18:38 +0000 2015
673370945936080896
Sun Dec 06 05:02:22 +0000 2015
67336685

In [75]:
pd.DataFrame(tweets).sort_values("created_at")

,created_at,favorite_count,is_retweet,retweeted_count,tw_id,url,user_favourites_count,user_followers_count,user_friends_count,user_listed_count,user_statuses_count
0,2015-12-05 12:41:02,39,0,47,673074590814486528,https://tjournal.ru/p/lesya-echo-off,140,253898,675,1173,40267
1,2015-12-05 15:50:23,42,0,58,673122243686375424,https://tjournal.ru/p/lesya-echo-off,5,893560,304,3232,93333
99,2015-12-05 15:52:39,0,1,0,673122814560440320,https://tjournal.ru/p/lesya-echo-off,269,81,57,3,8098
98,2015-12-05 15:52:55,0,1,0,673122881639989248,https://tjournal.ru/p/lesya-echo-off,87,47,144,0,903
97,2015-12-05 15:52:57,0,0,0,673122887876935680,https://tjournal.ru/p/lesya-echo-off,494,184,243,3,16472
96,2015-12-05 15:53:32,0,0,0,673123034430054400,https://tjournal.ru/p/lesya-echo-off,3,44,174,0,1740
95,2015-12-05 15:56:18,0,1,0,673123734165823488,https://tjournal.ru/p/lesya-echo-off,2379,180,176,5,26646
94,2015-12-05 15:57:32,0,1,0,673124043055349760,https://tjournal.ru/p/lesya-echo-off,70,713,755,4,7469
93,2015-12-05 15:59:18,0,1,0,673124487785816064,https://tjournal.ru/p/lesya-echo-off,227,106,637,1,1455
92,2015-12-05 15:59:25,0,1,0,673124518819426304,https://tjournal.ru/p/lesya-echo-off,1762,317,424,1,18223


In [55]:
sorted(tweets, key=lambda k: k['created_at']) 

[{'created_at': '2015-12-05 12:41:02',
  'favorite_count': 39,
  'is_retweet': 0,
  'retweeted_count': 47,
  'tw_id': 673074590814486528,
  'url': 'https://tjournal.ru/p/lesya-echo-off',
  'user_favourites_count': 140,
  'user_followers_count': 253888,
  'user_friends_count': 675,
  'user_listed_count': 1173,
  'user_statuses_count': 40267},
 {'created_at': '2015-12-05 12:48:49',
  'favorite_count': 37,
  'is_retweet': 0,
  'retweeted_count': 72,
  'tw_id': 673076548992397312,
  'url': 'https://tjournal.ru/p/lesya-echo-off',
  'user_favourites_count': 647,
  'user_followers_count': 213690,
  'user_friends_count': 750,
  'user_listed_count': 2437,
  'user_statuses_count': 125642},
 {'created_at': '2015-12-05 15:52:39',
  'favorite_count': 0,
  'is_retweet': 1,
  'retweeted_count': 0,
  'tw_id': 673122814560440320,
  'url': 'https://tjournal.ru/p/lesya-echo-off',
  'user_favourites_count': 269,
  'user_followers_count': 81,
  'user_friends_count': 57,
  'user_listed_count': 3,
  'user_st

In [ ]:
date = datetime.now()

In [ ]:
d = datetime.now() - timedelta(hours=2)

In [ ]:
d = str(d).split('.')[0]

In [ ]:
import json

In [ ]:
text = requests.get("https://api.vc.ru/1/paper").text

In [ ]:
json_req = json.loads(text)

In [ ]:
page = html.parse(urlopen("https://vc.ru/p/interview-it"))
root = page.getroot()
print root

In [ ]:
page = html.parse(urlopen("https://vc.ru/n/microsoft-store-russia"))
root = page.getroot()

# заголовок
title = root.find_class("b-article__head")
title = title[0].find("h1").text
print title

# парсим количество просмотров
view = root.get_element_by_id("hitsCount").text
view = view.replace(" ", "")
view = int(view)
print view

# Количество комментариев
comments = root.find_class("ccount")[0].text
comment = int(comments.replace(" ", ""))
print comment

tags = root.find_class("b-tags__tag")
tag_list = []
for tag in tags:
    tag_list.append(tag.text)
    
print tag_list

In [ ]:
print title

In [ ]:
vc = VCLoader()

In [ ]:
vc.get_link_info("https://vc.ru/n/microsoft-store-russia")

In [ ]:
vc_dict = vc.get_tj_news_info()

Вопросы:

1) Что делать с соурсе? Эти новости никто не репостит с TJournal, а от других источников их можно найти

2) Урлы для других запросов на VC. Не хотелось бы их брудфорсить.